## NeurAs

In [ ]:
config={
    "n_hidden_layers": 9,
    "enc_neurons": 256,
    "dec_layers": 7,
    "dec_neurons": 128,
    "epochs": 100,
    "batch_size": 5000,
    "L": 10,
    "learning_rate": 1e-4,
    "dataset": "2azQ1b91cZZ",
    "voxel_size": 0.008
    }

In [ ]:
command_args = ['-type', 'conf',
                '-i', '<path_to_conf_file>',
                '-filter', '<room_id>']

In [ ]:
import argparse
import os
import pickle
import time
import copy

import cv2
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import pye57
import torch
import torch.nn.functional as F
from numba import jit
from scipy.spatial.transform import Rotation
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

In [ ]:
class encMLP(nn.Module):
    def __init__(self, hidden_dim=128, embedding_dim_p=10, embedding_dim_c=4):   
        super(encMLP, self).__init__()

        self.embedding_dim_p = embedding_dim_p
        self.embedding_dim_c = embedding_dim_c
        
        self.block1 = nn.Sequential(nn.Linear(embedding_dim_p * 6 + 3, hidden_dim), nn.ReLU(),
                                    nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),
                                    nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),
                                    nn.Linear(hidden_dim, hidden_dim), nn.ReLU(), )

        self.block2 = nn.Sequential(nn.Linear(embedding_dim_p * 6 + 3 + hidden_dim, hidden_dim), nn.ReLU(),
                                    nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),
                                    nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),
                                    nn.Linear(hidden_dim, hidden_dim), )

        self.block3 = nn.Sequential(nn.Linear(embedding_dim_c * 6 + hidden_dim + 3, hidden_dim // 2), nn.ReLU(), )

        self.block4 = nn.Sequential(nn.Linear(hidden_dim // 2, 3), nn.Sigmoid(), )

    @staticmethod
    def positional_encoding(x, L):
        out = [x]
        for j in range(L):
            out.append(torch.sin(2 ** j * x))
            out.append(torch.cos(2 ** j * x))
        return torch.cat(out, dim=1)

    def forward(self, p, c):
        emb_p = self.positional_encoding(p, self.embedding_dim_p)
        emb_c = self.positional_encoding(c, self.embedding_dim_c)
        
        b1 = self.block1(emb_p)
        
        b2 = self.block2(torch.cat((b1, emb_p), dim=1))
        
        b3 = self.block3(torch.cat((b2, emb_c), dim=1))
        
        b4 = self.block4(b3)
        
        return b4

In [ ]:
class decMLP(nn.Module):

    def __init__(self, hidden_size, n_layers):
        super().__init__()

        self.input_size = 5
        self.output_size = 3
        
        self.input_layer = nn.Linear(self.input_size, hidden_size)

        self.hidden_layers = nn.ModuleList([
            nn.Linear(hidden_size, hidden_size) for _ in range(n_layers)
        ])

        self.output_layer = nn.Linear(hidden_size, self.output_size)


    def forward(self, x):
        
        x = F.relu(self.input_layer(x))
        
        for layer in self.hidden_layers:
            x = F.relu(layer(x))

        x = torch.sigmoid(self.output_layer(x))
        return x
    

In [ ]:
def get_cameras_data_from_conf(conf_file_path, scan_filter=None):
    working_dir = os.path.dirname(conf_file_path)
    cameras_data = {}
    header_info = ''

    data = open(conf_file_path).readlines()

    for line in tqdm(data):

        # split by spaces
        content = line.split()

        if not content:
            continue

        # header information
        if content[0] == 'dataset':
            title = content[1] + " dataset"
            header_info += "\n#####" + "#" * len(title) + "#####"
            header_info += "\n#    " + title + "    #"
            header_info += "\n#####" + "#" * len(title) + "#####\n"

        if content[0] == 'n_images':
            header_info += "\nContains " + content[1] + " captures"

        if content[0] == 'depth_directory':
            depth_dir = content[1]
            header_info += "\nDepth directory is " + depth_dir

        if content[0] == 'color_directory':
            rgb_dir = content[1]
            header_info += "\nRGB directory is " + rgb_dir

        # scan data
        if content[0] == 'intrinsics_matrix':
            intrinsics_list = content[1:]
            # intrinsics matrix
            K = np.zeros((3, 3))
            K[0, :] = intrinsics_list[0:3]
            K[1, :] = intrinsics_list[3:6]
            K[2, :] = intrinsics_list[6:9]

        if content[0] == 'scan':
            depth_file_name = content[1]
            rgb_file_name = content[2]
            extrinsics_list = content[3:]

            if scan_filter:
                if depth_file_name.split('_')[0] not in scan_filter:
                    continue

            scan_depth_image = o3d.io.read_image(working_dir + '/' + depth_dir + '/' + depth_file_name)
            # 0.25mm per unit, divide by 4000 to get meters
            scan_depth_image = np.asarray(scan_depth_image, dtype=np.float32) / 4000

            image = o3d.io.read_image(working_dir + '/' + rgb_dir + '/' + rgb_file_name)
            image = np.asarray(image)

            height, width, _ = image.shape

            # extrinsics matrix
            cam_matrix = np.zeros((4, 4))
            cam_matrix[0, :] = extrinsics_list[0:4]
            cam_matrix[1, :] = extrinsics_list[4:8]
            cam_matrix[2, :] = extrinsics_list[8:12]
            cam_matrix[3, :] = extrinsics_list[12:16]

            opengl_to_opencv = np.zeros((4, 4))
            opengl_to_opencv[0, 0] = 1
            opengl_to_opencv[1, 1] = -1
            opengl_to_opencv[2, 2] = -1
            opengl_to_opencv[3, 3] = 1
            cam_matrix = np.dot(cam_matrix, opengl_to_opencv)

            cameras_data[rgb_file_name.split('.')[0]] = {'extrinsics': cam_matrix,
                                                         'intrinsics': K,
                                                         'height': height,
                                                         'width': width,
                                                         'imageBGR': cv2.cvtColor(image, cv2.COLOR_RGB2BGR),
                                                         'scan_depth_image': scan_depth_image
                                                         }
    print(header_info)
    return cameras_data

In [ ]:
def get_cameras_data(e57):
    cameras_data = {}

    imf = e57.image_file
    root = imf.root()

    for image_idx, image2D in enumerate(tqdm(root['images2D'])):
        # get extrinsic matrix
        tx = image2D['pose']['translation']['x'].value()
        ty = image2D['pose']['translation']['y'].value()
        tz = image2D['pose']['translation']['z'].value()

        t = np.array([tx, ty, tz])

        rx = image2D['pose']['rotation']['x'].value()
        ry = image2D['pose']['rotation']['y'].value()
        rz = image2D['pose']['rotation']['z'].value()
        rw = image2D['pose']['rotation']['w'].value()

        r = Rotation.from_quat(np.array([rx, ry, rz, rw]))

        cam_matrix = np.zeros((4, 4))
        cam_matrix[3, 3] = 1
        cam_matrix[:-1, -1] = t
        cam_matrix[:3, :3] = r.as_matrix()

        opengl_to_opencv = np.zeros((4, 4))
        opengl_to_opencv[0, 0] = 1
        opengl_to_opencv[1, 1] = -1
        opengl_to_opencv[2, 2] = -1
        opengl_to_opencv[3, 3] = 1
        cam_matrix = np.dot(cam_matrix, opengl_to_opencv)

        # get intrinsic matrix
        pinhole = image2D['pinholeRepresentation']

        focal_length = pinhole['focalLength'].value()
        pixel_height = pinhole['pixelHeight'].value()
        pixel_width = pinhole['pixelWidth'].value()
        principal_point_x = pinhole['principalPointX'].value()
        principal_point_y = pinhole['principalPointY'].value()

        K = np.zeros((3, 3))
        K[2, 2] = 1
        K[0, 0] = focal_length / pixel_width
        K[1, 1] = focal_length / pixel_height
        K[0, 2] = principal_point_x
        K[1, 2] = principal_point_y

        # get picture from blob
        jpeg_image = pinhole['jpegImage']
        jpeg_image_data = np.zeros(shape=jpeg_image.byteCount(), dtype=np.uint8)
        jpeg_image.read(jpeg_image_data, 0, jpeg_image.byteCount())
        image = cv2.imdecode(jpeg_image_data, cv2.IMREAD_COLOR)

        height, width, channels = image.shape

        ##############################################
        # generate depth image from laser scan
        pc_data = e57.read_scan(image_idx // 6)
        pts = [pc_data['cartesianX'], pc_data['cartesianY'], pc_data['cartesianZ']]
        scan_pcd = o3d.geometry.PointCloud()
        scan_pcd.points = o3d.utility.Vector3dVector(np.vstack(pts).transpose())

        renderer = o3d.visualization.rendering.OffscreenRenderer(width, height)
        mtl = o3d.visualization.rendering.MaterialRecord()
        mtl.base_color = [1.0, 1.0, 1.0, 1.0]  # RGBA
        mtl.shader = "defaultUnlit"
        renderer.scene.add_geometry("textured_mesh", scan_pcd, mtl)
        o3d_intrinsics = o3d.camera.PinholeCameraIntrinsic(width, height, K[0, 0], K[1, 1], K[0, 2], K[1, 2])
        renderer.setup_camera(o3d_intrinsics, np.linalg.inv(cam_matrix))
        scan_depth_image = np.array(renderer.render_to_depth_image(z_in_view_space=True))

        cameras_data[image_idx] = {'extrinsics': cam_matrix,
                                   'intrinsics': K,
                                   'height': height,
                                   'width': width,
                                   'imageBGR': image,
                                   'scan_depth_image': scan_depth_image,
                                   'scan_idx': image_idx // 6
                                   }

    return cameras_data


@jit(nopython=True)
def project_to_camera(intrinsic_matrix, distortion, width, height, pts):
    _, n_pts = pts.shape

    pixs = np.zeros((2, n_pts), dtype=np.float64)

    k1, k2, p1, p2, k3 = distortion
    # fx, _, cx, _, fy, cy, _, _, _ = intrinsic_matrix
    # print('intrinsic=\n' + str(intrinsic_matrix))
    fx = intrinsic_matrix[0, 0]
    fy = intrinsic_matrix[1, 1]
    cx = intrinsic_matrix[0, 2]
    cy = intrinsic_matrix[1, 2]

    x = pts[0, :]
    y = pts[1, :]
    z = pts[2, :]

    xl = np.divide(x, z)
    yl = np.divide(y, z)
    r2 = xl ** 2 + yl ** 2
    xll = xl * (1 + k1 * r2 + k2 * r2 ** 2 + k3 * r2 ** 3) + 2 * p1 * xl * yl + p2 * (r2 + 2 * xl ** 2)
    yll = yl * (1 + k1 * r2 + k2 * r2 ** 2 + k3 * r2 ** 3) + p1 * (r2 + 2 * yl ** 2) + 2 * p2 * xl * yl
    pixs[0, :] = fx * xll + cx
    pixs[1, :] = fy * yll + cy

    # compute mask of valid projections
    valid_z = z > 0
    valid_xpix = np.logical_and(pixs[0, :] >= 0, pixs[0, :] < width)
    valid_ypix = np.logical_and(pixs[1, :] >= 0, pixs[1, :] < height)
    valid_pixs = np.logical_and(valid_z, np.logical_and(valid_xpix, valid_ypix))
    return pixs, valid_pixs


@jit(nopython=True)
def filter_points_in_view(range_sparse, dists, width, height, indexes):
    depth_image = np.full((height, width), fill_value=-1, dtype=np.float64)
    result = np.full((height, width), fill_value=-1, dtype=np.int32)

    for idx, dist in enumerate(dists):
        x, y = range_sparse[idx]

        # if a point closer to the camera has been projected to this pixel
        if depth_image[y, x] != -1 and depth_image[y, x] < dist:
            continue

        depth_image[y, x] = dist

        result[y, x] = indexes[idx]

    return depth_image, result


def get_viewpoint_depth(pts_hom, cam_matrix, K, width, height):
    """
    Get a depth image with points visible from a camera viewpoint and their corresponding indexes.

    Parameters:
    - pts_hom (array-like): A vector of 3D points to project onto the camera view.
    - cam_matrix (array-like): The extrinsic matrix of the camera.
    - K (array-like): The intrinsic matrix of the camera.
    - width (int): The width of the output depth image (in pixels).
    - height (int): The height of the output depth image (in pixels).

    Returns:
    - depth_image (array-like): A matrix with depth values at each (x, y) pixel location.
    - point_indexes (array-like): A matrix with indices of the corresponding points at each (x, y) pixel location.

    Notes:
    - Ensure that the input points in 'pts_hom' are in a format compatible with the camera's coordinate system and that
    'cam_matrix' and 'K' are properly calibrated camera matrices.
    - The 'depth_image' will contain depth values for visible points at each pixel location.
    - The 'point_indexes' will contain the index of the corresponding point from 'pts_hom' at each pixel location.
    """
    # project points to camera coordinate system
    pts_in_camera = np.dot(np.linalg.inv(cam_matrix), pts_hom)

    pixels, valid_pixels = project_to_camera(K,
                                             np.array([0, 0, 0, 0, 0]),
                                             width,
                                             height,
                                             pts_in_camera)

    range_sparse = np.transpose(np.vstack((pixels[0, valid_pixels],
                                           pixels[1, valid_pixels])).astype(int))
    dists = pts_in_camera[2, valid_pixels]
    indexes = np.where(valid_pixels)[0]

    depth_image, point_indexes = filter_points_in_view(range_sparse, dists, width, height, indexes)

    return depth_image, point_indexes


def compute_pairwise_mappings(idxs_per_camera, sparse_per_camera):
    camera_mappings = {}
    for i in tqdm(range(len(idxs_per_camera) - 1)):
        for j in range(i + 1, len(idxs_per_camera)):
            # find points of cam i in cam j
            np_mappings = np.intersect1d(idxs_per_camera[i], idxs_per_camera[j], return_indices=True)

            # list of corresponding indices for i and j
            mappings_i = sparse_per_camera[i][np_mappings[1]]
            mappings_j = sparse_per_camera[j][np_mappings[2]]

            # store mappings in data structures
            camera_mappings[(i, j)] = [mappings_i, mappings_j]
    return camera_mappings


In [ ]:
def create_tsdf(cameras_data, voxel_size):
    
    volume = o3d.pipelines.integration.ScalableTSDFVolume(voxel_length=voxel_size,
                                                          sdf_trunc=10 * voxel_size,
                                                          color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8)

    for camera in tqdm(cameras_data):
        image = o3d.geometry.Image(cv2.cvtColor(cameras_data[camera]['imageBGR'], cv2.COLOR_BGR2RGB))
        scan_depth_image = o3d.geometry.Image(cameras_data[camera]['scan_depth_image'] * 1000)

        rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(image,
                                                                  scan_depth_image,
                                                                  depth_trunc=10.0,
                                                                  convert_rgb_to_intensity=False)

        K = cameras_data[camera]['intrinsics']
        width = cameras_data[camera]['width']
        height = cameras_data[camera]['height']
        o3d_intrinsics = o3d.camera.PinholeCameraIntrinsic(width, height, K[0, 0], K[1, 1], K[0, 2], K[1, 2])

        volume.integrate(rgbd,
                         o3d_intrinsics,
                         np.linalg.inv(cameras_data[camera]['extrinsics']))

    return volume

### Load data

In [ ]:
ap = argparse.ArgumentParser()
ap.add_argument('-type', '--input_type', required=True, type=str, choices=['e57', 'conf'],
                help="Type of input files.\n"
                     "'e57' -  E57 file contains all the information.\n"
                     "'conf' - Config file containing camera parameters and path to depth and rgb.\n")
ap.add_argument('-i', '--input_path', required=True, type=str,
                help="Path to the input file.")
ap.add_argument('-cache', '--cache_path', required=False, help="Path for cache directory.")
ap.add_argument('-filter', '--region_filter', required=False, type=int, nargs='+',
                help="List of region ids to keep.")
args = vars(ap.parse_args(command_args))

if args['input_type'] == 'e57':
    print("\nLoading E57...")
    e57 = pye57.E57(args['input_path'])
    pcd = o3d.geometry.PointCloud()
    print("\nReading point cloud scans...")
    for scan_idx in tqdm(range(e57.scan_count)):
        pc_data = e57.read_scan(scan_idx, colors=False)
    
        pc_points = [pc_data['cartesianX'], pc_data['cartesianY'], pc_data['cartesianZ']]
        tmp_pcd = o3d.geometry.PointCloud()
        tmp_pcd.points = o3d.utility.Vector3dVector(np.vstack(pc_points).transpose())
    
        if args['cache_path'] and os.path.isfile(args['cache_path'] + '/normals/' + str(scan_idx) + '.npy'):
            # load normals
            loaded_array = np.load(args['cache_path'] + '/normals/' + str(scan_idx) + '.npy')
            tmp_pcd.normals = o3d.utility.Vector3dVector(loaded_array)
        else:
            # estimate normals and flip using scanner position
            tmp_pcd.estimate_normals()
            header = e57.get_header(scan_idx)
            cam_matrix = np.zeros((4, 4))
            cam_matrix[3, 3] = 1
            cam_matrix[:-1, -1] = header.translation
            cam_matrix[:3, :3] = header.rotation_matrix
            tmp_pcd.orient_normals_towards_camera_location(camera_location=cam_matrix[:3, 3])
    
            # cache normals
            if args['cache_path']:
                if not os.path.exists(args['cache_path']):
                    os.makedirs(args['cache_path'])
                    os.makedirs(args['cache_path'] + '/normals/')
                else:
                    if not os.path.exists(args['cache_path'] + '/normals/'):
                        os.makedirs(args['cache_path'] + '/normals/')
    
                    np.save(args['cache_path'] + '/normals/' + str(scan_idx) + '.npy',
                            np.array(tmp_pcd.normals))
        pcd += tmp_pcd
        # break
    
    pcd = pcd.voxel_down_sample(voxel_size=config["voxel_size"])

    # load camera data from cache if available
    print("\nLoading cameras data...")
    if args['cache_path'] and os.path.isfile(args['cache_path'] + '/camera_data.pickle'):
        with open(args['cache_path'] + '/camera_data.pickle', 'rb') as handle:
            data = pickle.load(handle)
    else:
        cameras_data = get_cameras_data(e57)
    
        # cache camera data
        if args['cache_path']:
            with open(args['cache_path'] + '/camera_data.pickle', 'xb') as handle:
                pickle.dump(cameras_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("\nDone")

elif args['input_type'] == 'conf':
    scan_filter = []
    if args['region_filter']:
        segmentation_path = os.path.dirname(args['input_path']) + '/panorama_to_region.txt'
        if not os.path.isfile(segmentation_path):
            print("\nERROR: " + segmentation_path + " not found!")
            print("Region segmentation is required to use filter. Exiting...")
            exit()

        seg_data = open(segmentation_path).readlines()

        print("\nProcessing region segmentation...")
        for line in tqdm(seg_data):

            # split by spaces
            content = line.split()

            if not content:
                continue

            if int(content[2]) in args['region_filter']:
                scan_filter.append(content[1])

    print("\nLoading camera data...")
    cameras_data = get_cameras_data_from_conf(args['input_path'], scan_filter)

    print("\nIntegrating data in TSDF...")
    tsdf = create_tsdf(cameras_data, config["voxel_size"])

    print("\nExtracting point cloud...")
    pcd = tsdf.extract_point_cloud()

    # file_name = "voxel_size_" + str(config["voxel_size"]) + "_batch_" + str(config["batch_size"]) + "_epochs_" + str(config["epochs"]) + "_L_" + str(config["L"]) + ".ply"
    # o3d.io.write_point_cloud(file_name, pcd)

    print("\nDone.")

### Setup mappings

In [ ]:
# # explicit filter
# image_list = [11, 41]
# cameras_data = {image_list.index(key): data[key] for key in image_list}

sparse_per_camera = np.empty(len(cameras_data), dtype=object)
idxs_per_camera = np.empty(len(cameras_data), dtype=object)

cam_pos = np.empty(len(cameras_data), dtype=object)
vector_src = np.empty(len(cameras_data), dtype=object)
vector_tgt = np.empty(len(cameras_data), dtype=object)

print("\nProjecting points...")
for cam_idx, camera in enumerate(tqdm(cameras_data)):
    
    # project points to camera
    pts_hom = np.array(pcd.points).T
    pts_hom = np.vstack([pts_hom, np.ones(len(pts_hom[1]), dtype=float)])
    depth_image, point_indexes = get_viewpoint_depth(pts_hom,
                                                     cameras_data[camera]['extrinsics'],
                                                     cameras_data[camera]['intrinsics'],
                                                     cameras_data[camera]['width'],
                                                     cameras_data[camera]['height'])

    # z-buffering filter
    zbuffer_invalid = np.where(abs(depth_image - cameras_data[camera]['scan_depth_image']) > 0.06)
    point_indexes[zbuffer_invalid] = -1

    # get the row,col of points that were actually projected
    sparse = np.array(list(zip(np.where(point_indexes != -1)))).T.reshape(-1, 2)
    sparse_per_camera[cam_idx] = sparse
    
    # keep index of 3D points that were actually projected
    idxs_per_camera[cam_idx] = point_indexes[point_indexes != -1]
    
    # get colour data
    img_bgr = cameras_data[camera]['imageBGR']

    vector_src[cam_idx] = np.array(pcd.points)[idxs_per_camera[cam_idx]]    # X Y Z
    
    if len(vector_src[cam_idx]) == 0:
        cam_pos[cam_idx] = np.array(pcd.points)[idxs_per_camera[cam_idx]]
    else:
        pos = cameras_data[camera]['extrinsics'][:3, 3]
        cam_pos[cam_idx] = np.vstack([pos] * len(vector_src[cam_idx]))    # Tx Ty Tz

    vector_tgt[cam_idx] = img_bgr[sparse[:, 0], sparse[:, 1]] / 255    # B G R
    
vector_src = np.concatenate(vector_src)
cam_pos = np.concatenate(cam_pos)
vector_tgt = np.concatenate(vector_tgt)

# z-score normalization
input_mean = np.mean(vector_src, axis=0)
input_std = np.std(vector_src, axis=0)
norm_vector_src = (vector_src - input_mean) / input_std

cam_pos_mean = np.mean(cam_pos, axis=0)
cam_pos_std = np.std(cam_pos, axis=0)
norm_cam_pos = (cam_pos - cam_pos_mean) / cam_pos_std

norm_inputs = torch.tensor(np.hstack([norm_vector_src, norm_cam_pos]))

### Training scene encoder

In [ ]:
start = time.time()

torch.manual_seed(42)

enc_dataset = TensorDataset(norm_inputs, torch.tensor(vector_tgt))
dec_trainloader = DataLoader(enc_dataset, batch_size=config["batch_size"], shuffle=True, num_workers=4, pin_memory=True)

mlp = encMLP(config["enc_neurons"], config["L"])

mlp = mlp.to("cuda", dtype=torch.float32)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=config["learning_rate"])

for epoch in range(config['epochs']):

    print(f"\nEpoch {epoch + 1} / {config['epochs']}:")

    cost = 0.0
    n_batches = 0
    for batch in tqdm(dec_trainloader):
        inputs, targets = batch

        optimizer.zero_grad()
        
        outputs = mlp(inputs[:, :3].to("cuda", dtype=torch.float32), 
                      inputs[:, -3:].to("cuda", dtype=torch.float32))

        loss = loss_function(outputs, targets.to("cuda", dtype=torch.float32))

        loss.backward()

        optimizer.step()

        cost += loss.item()
        n_batches += 1
        
        # wandb.log({"Batch": n_batches * (epoch + 1),
        #            "Loss": loss.item()})

    print("Average cost for Epoch: " + str(cost / n_batches))
    # wandb.log({"Epoch": epoch,        
    #            "Average Loss": cost / n_batches})

    torch.save(mlp, 'v_d_encoder.pth')
    
end = time.time()
seconds = int(end - start)
print("\n---------------------------------")
print(f"\nCompleted in "f"{seconds // 3600:02d}:"f"{(seconds // 60) % 60:02d}:"f"{seconds % 60:02d}")


### Creating encoder vis

In [ ]:
mlp = torch.load('v_d_encoder.pth')
mlp.to("cuda", dtype=torch.float32)

norm_src_test = (np.array(pcd.points) - input_mean) / input_std

for idx, camera_id in enumerate(cameras_data):

    if not idx == 42:
        continue
    
    cam_matrix = cameras_data[camera_id]['extrinsics']
    K = cameras_data[camera_id]['intrinsics']
    width, height = cameras_data[camera_id]['width'], cameras_data[camera_id]['height']
    
    pos = copy.deepcopy(cam_matrix[:3, 3])
    
    cam_pos_test = np.vstack([pos] * len(norm_src_test))
    norm_cam_pos_test = (cam_pos_test - cam_pos_mean) / cam_pos_std
    
    norm_inputs_test =  torch.tensor(np.hstack([norm_src_test, norm_cam_pos_test]))
    # norm_inputs_test =  torch.tensor(norm_src_test)
    
    test_batch_size = 10000
    test_dataset = TensorDataset(norm_inputs_test)
    testloader = DataLoader(test_dataset, batch_size=test_batch_size, num_workers=4)
    
    mlp.eval()
    
    all_predictions = []
    
    with torch.no_grad():
        
        for batch in tqdm(testloader):
    
            batch_predictions = mlp(batch[0][:, :3].to("cuda", dtype=torch.float32), 
                                    batch[0][:, -3:].to("cuda", dtype=torch.float32))
    
            all_predictions.append(batch_predictions)
    
    final_predictions = torch.cat(all_predictions, dim=0).to("cpu")
    
    pcd.colors = o3d.utility.Vector3dVector(final_predictions.numpy()[:, ::-1])
    
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)
    sphere.translate(pos)
    sphere.paint_uniform_color([1,0,0])
    o3d.visualization.draw_geometries([pcd, sphere])
    
    # # render image
    # renderer = o3d.visualization.rendering.OffscreenRenderer(width, height)
    # mtl = o3d.visualization.rendering.MaterialRecord()
    # mtl.base_color = [1.0, 1.0, 1.0, 1.0]  # RGBA
    # mtl.shader = "defaultUnlit)"
    # renderer.scene.add_geometry("textured_mesh", pcd, mtl)
    # eye = np.array([ -4.6261574364973619, 0.64065648187804303, 1.6956380986471207 ])
    # center = eye - np.array([ -0.67223291846430644, -0.73171576421941253, 0.11267184087322939 ])
    # up = np.array([ 0.076496362433848372, 0.082724737828267197, 0.99363218762559902 ])
    # renderer.setup_camera(90.0, center, eye, up)
    # view_dep_image = np.array(renderer.render_to_image())

    # cv2.imshow('view_dep_image', cv2.cvtColor(view_dep_image, cv2.COLOR_RGB2BGR))
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    
    file_name = "voxel_size_" + str(config["voxel_size"]) + "_batch_" + str(config["batch_size"]) + "_epochs_" + str(config["epochs"]) + "_L_" + str(config["L"]) + ".ply"
    o3d.io.write_point_cloud(file_name, pcd)


### Scene decoders

In [ ]:
mlp = torch.load('v_d_encoder.pth')
mlp.to("cuda", dtype=torch.float32)
file_out_name = os.path.basename(args['input_path'].split('.')[0])
reference_cam = 24
batch_size = 1000
epochs = 20

for camera_idx, camera_key in enumerate(cameras_data):

    print("Camera " + str(camera_idx))

    camera = cameras_data[camera_key]

    # depth_image = camera['scan_depth_image']
    # height, width = camera['height'], camera['width']
    # K = camera['intrinsics']
    # extrinsics = camera['extrinsics']
    # o3d_intrinsics = o3d.camera.PinholeCameraIntrinsic(width, height, K[0, 0], K[1, 1], K[0, 2], K[1, 2])
    # point_cloud = o3d.geometry.PointCloud.create_from_depth_image(o3d.geometry.Image(depth_image), o3d_intrinsics, np.linalg.inv(extrinsics))
    # point_cloud.remove_non_finite_points()

    point_cloud = pcd

    # project points to camera
    pts_hom = np.array(point_cloud.points).T
    pts_hom = np.vstack([pts_hom, np.ones(len(pts_hom[1]), dtype=float)])
    depth_image, point_indexes = get_viewpoint_depth(pts_hom,
                                                     camera['extrinsics'],
                                                     camera['intrinsics'],
                                                     camera['width'],
                                                     camera['height'])
    
    # Z buffering filter
    zbuffer_invalid = np.where(abs(depth_image - camera['scan_depth_image']) > 0.06)
    point_indexes[zbuffer_invalid] = -1

    # print(len(point_indexes[point_indexes != -1]))

    # check if there are enough valid points
    if len(point_indexes[point_indexes != -1]) < 100:
        print("Not enough geometric data...")

    # get the row,col of points that were actually projected
    sparse = np.array(list(zip(np.where(point_indexes != -1)))).T.reshape(-1, 2)

    # get colour data
    img_bgr = camera['imageBGR']

    enc_vector_src = np.array(point_cloud.points)[point_indexes[point_indexes != -1]]    # X Y Z
    enc_vector_tgt = img_bgr[sparse[:, 0], sparse[:, 1]] / 255         # B G R

    vector_src_test =  torch.tensor((enc_vector_src - input_mean) / input_std)

    # camera position
    pos = cameras_data[list(cameras_data.keys())[reference_cam]]['extrinsics'][:3, 3]
    
    cam_pos_test = np.vstack([pos] * len(vector_src_test))
    norm_cam_pos_test = (cam_pos_test - cam_pos_mean) / cam_pos_std
    norm_inputs_test =  torch.tensor(np.hstack([vector_src_test, norm_cam_pos_test]))

    test_batch_size = 10000
    test_dataset = TensorDataset(norm_inputs_test)
    testloader = DataLoader(test_dataset, batch_size=test_batch_size, num_workers=4, pin_memory=True)

    mlp.eval()

    all_predictions = []

    print("Getting data from scene encoder...")
    with torch.no_grad():

        for batch in tqdm(testloader):

            batch_predictions = mlp(batch[0][:, :3].to("cuda", dtype=torch.float32), 
                                    batch[0][:, -3:].to("cuda", dtype=torch.float32))

            all_predictions.append(batch_predictions)

    final_predictions = torch.cat(all_predictions, dim=0).to("cpu")

    # now train another MLP to learn RGBXY -> RGB

    torch.manual_seed(42)

    # BGRYX vectors
    BGRYX_src = np.hstack((enc_vector_tgt,
                            sparse[:, 0:1] / camera['height'],
                            sparse[:, 1:2] / camera['width']))

    tensor_x = torch.Tensor(BGRYX_src)
    tensor_y = final_predictions

    my_dataset = TensorDataset(tensor_x, tensor_y)
    trainloader = DataLoader(my_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

    # initialize scene decoder
    decoder = decMLP(config["dec_neurons"], config["dec_layers"])
    
    decoder = decoder.to("cuda", dtype=torch.float32)
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(decoder.parameters(), lr=config["learning_rate"])
    
    losses = []
    print("\nTraining scene decoder...")
    for epoch in range(epochs):

        print(f"\nEpoch {epoch + 1} / {epochs}:")

        cost = 0.0
        n_batches = 0

        for data in tqdm(trainloader):
            inputs, targets = data

            optimizer.zero_grad()

            batch_t = inputs.to("cuda", dtype=torch.float32)

            outputs = decoder(batch_t)

            loss = loss_function(outputs, targets.to("cuda", dtype=torch.float32))

            loss.backward()

            optimizer.step()

            cost += loss.item()
            n_batches += 1

        print("Average cost for Epoch: " + str(cost / n_batches))
        losses.append(cost / n_batches)

    # # Plotting the loss graph
    # plt.plot(range(1, epochs + 1), losses, label='Training Loss')
    # plt.xlabel('Epochs')
    # plt.ylabel('Loss')
    # plt.title('Training Loss Over Epochs')
    # plt.legend()
    # plt.show()

    #########################################################
    # testing the scene decoder

    source_test_img = camera['imageBGR'] / 255
    height, width = camera['height'], camera['width']
    vector_src_test = np.hstack((source_test_img.reshape(-1, 3),
                                 np.indices((height, width)).reshape(2, -1).T / np.array([height, width])))

    tensor_x_test = torch.tensor(vector_src_test)

    f_batch_size = 10000
    f_test_dataset = TensorDataset(tensor_x_test)
    f_testloader = DataLoader(f_test_dataset, batch_size=f_batch_size, num_workers=4, pin_memory=True)

    decoder.eval()

    print("\nGenerating enhanced texture...")
    with torch.no_grad():

        all_predictions = []

        for batch in tqdm(f_testloader):

            batch_t = batch[0].to("cuda", dtype=torch.float32)

            predictions = decoder(batch_t)

            all_predictions.append(predictions)

    final_predictions = torch.cat(all_predictions, dim=0).to("cpu")

    synthetic_img = final_predictions.view((height, width, 3)).numpy()

    cv2.imwrite(file_out_name + '_' + str(camera_idx) + '.png', (synthetic_img * 255).astype(int))

    plt.imshow(synthetic_img[:, :, ::-1])
    plt.axis('off')
    plt.tight_layout()
    plt.show()